In [2]:
%run data_preprocessing.ipynb

ERROR:root:File `'data_preprocessing.ipynb.py'` not found.


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
# from simpletransformers.classification import ClassificationModel
# from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import logging
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import ast

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# from google.colab import files
# uploaded = files.upload()

In [92]:
multi_label_classification = pd.read_csv("/content/multilabel_classification", usecols=["text", "labels"])
multi_label_classification.head()

,text,labels
0,The districts of Guidan-Roumji in Niger and Do...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. ...
1,The week 6 was marked by a sudden apparition o...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. ...
2,Spatial analysis shows an epidemic area common...,[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. ...
3,During week 14 it is noted that: Burkina Faso:...,[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. ...
4,Burkina Faso: Titao was the last district in a...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. ...


In [0]:
# summary_state = summary_state.drop(columns=["State", "Count"])
# summary_state = summary_state.rename(columns={"Summary":"text", "Label":"labels"})
# summary_state = summary_state.reset_index(drop=True)
multi_label_classification = multi_label_classification.reset_index()

In [4]:
separation = int(0.8 * len(multi_label_classification))
train_data = multi_label_classification[:separation]
test_data = multi_label_classification[separation:]
print(len(test_data))
train_data

32


,index,text,labels
0,0,The districts of Guidan-Roumji in Niger and Do...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. ...
1,1,The week 6 was marked by a sudden apparition o...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. ...
2,2,Spatial analysis shows an epidemic area common...,[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. ...
3,3,During week 14 it is noted that: Burkina Faso:...,[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. ...
4,4,Burkina Faso: Titao was the last district in a...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. ...
...,...,...,...
123,123,From week 40 to 43 (corresponding to October 2...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. ...
124,124,From week 44 to 47 (corresponding to November ...,[0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. ...
125,125,From week 48 to 52 (corresponding to December ...,[0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. ...
126,126,The first week of the year 2017 is marked by t...,[0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. ...


In [0]:
# model = ClassificationModel('roberta', 'roberta-base', num_labels=3, 
#                             use_cuda=False, args={"train_batch_size": 1 ,
#                                                   'reprocess_input_data': True, 'overwrite_output_dir': True})

model = MultiLabelClassificationModel('roberta', 'roberta-base', use_cuda=False, 
                                      num_labels=20, args={'reprocess_input_data': True,
                                                           'overwrite_output_dir': True, 'num_train_epochs': 3})



In [0]:
model.train_model(train_data)

In [0]:
#micro average takes into account true positives, false negatives, and false positives
def f1_multiclass(labels, preds):
    print(labels)
    preds = preds>0.2
    print(preds)
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(test_data, f1=f1_multiclass,
                                                           roc=roc_auc_score)

In [0]:
print(result)

The model achieves 87.5% accuracy with multiclass classification - for each report, sum the total number of states (0 1 or 2) and divide by the number of countries for which data is recorded.

For multilabel classification (with random truncation)- (predicting the state of every country from the text):
1. We will use the ROC-AUC evaluation metric which tells us for each label how good the model is at discriminating between classes (i.e. how good is it at predicting 0s as 0s and 1s as 1s)
F1 = 0.702
ROC = 0.5487

Testing multilabel classification with a sliding window

The results seem to be significantly worse for multilabel classification. My hypothesis for this is since BERT has a maximum number of tokens and much of the text is often larger than this limit, when it cuts out text to reduce it to the maximum size (i.e. by sampling from the middle) information is being lost which the model can't learn from. For example, if a part of the sentence cut out is one that talks about how bad the epidemic was in Burkina Faso, the model will assume a state of 0.

Solution: LongFormer

Model

LongFormer -> Multilabel Classification (using either scikit-learn's multioutput classifier and fitting a logistic algorithm to each label or a simple affine layer with a sigmoid activation function)]


*NOTE - in the future install transformers by running pip install git+https://github.com/huggingface/transformers in venv - i.e. installing directly from github and not from the latest release* 

In [6]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qjj7hoec
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-qjj7hoec
     |████████████████████████████████| 3.8MB 7.8MB/s 
     |████████████████████████████████| 1.1MB 51.2MB/s 
     |████████████████████████████████| 890kB 54.5MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=666275 sha256=9803e96b4877bbb73f6327141510e744db26c5ba56bb472316e61d7289ad75bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-hh_v5xuu/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=718c924c13877ff2507561cd0a0f33a0e4330769e736bb498f7ae7d4f69539be
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [5]:
from transformers import LongformerModel, LongformerTokenizer
import numpy as np
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerModel.from_pretrained('allenai/longformer-base-4096')
model.cuda()

LongformerModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): BertSelfOutpu

In [6]:
torch.cuda.empty_cache()
#note when encoded the start of every token is added with an <s> for classification purposes
input_ids = tokenizer.encode("Hello, my dog is cute")  # Batch size 1
print(tokenizer.decode(input_ids))
print(tokenizer.decode(input_ids[0]))

Thu Jun  4 07:40:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    32W /  70W |   1515MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [11]:
tokenized = multi_label_classification["text"]\
                                    .apply(lambda x: tokenizer.encode(x, add_special_tokens_tokens=True))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)
#padded according to max_length so we can feed the model in batches - padding works as such, add 0s after the length
#of the original sequence l such that the new length is = max_len = 2530
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])


2530


In [42]:
np.array(padded.shape)
#Add masking so that know BERT knows where to attend to => so it can ignore padding
#adds a 1 everwhere where the condition of padded != 0 is satisfied, in other words everywhere there isn't padding
attention_mask = np.where(padded != 0, 1, 0)
#add global attention represented with 2 - for now just add 1 at the classification <s> token
attention_mask[:, [0, -1]] = 2
print(attention_mask)
device = torch.device("cuda")

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)

def create_save_feature_representation():
  first_time = True
  for i in range(0,len(input_ids),2):
    with torch.no_grad():
      predictions, hidden_states = model(input_ids[i:i+2], attention_mask=attention_mask[i:i+2])
      scores = predictions[:,0,:].cpu().numpy()
      if not first_time:
        curr = np.load("lang_features.npy")
        scores = np.concatenate((curr, scores))
      else:
        first_time = False
      np.save("lang_features.npy",scores)

create_save_feature_representation()

[[2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]
 ...
 [2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]]


In [0]:
features = np.load("lang_features.npy")
#convert the labels into numpy arrays since they are stored as string representations
multi_label_classification["labels"] = multi_label_classification["labels"].apply(lambda x: np.fromstring(x[1:-1], dtype=np.float32, sep=' '))

In [0]:
train_features, test_features = features[:separation], features[separation:]
hidden_size = features.shape[1]

#convert to float datatype that can be then cast to a tensor in PyTorch
labels = np.array([label for label in np.array(multi_label_classification["labels"].values)])
train_labels, test_labels = labels[:separation], labels[separation:]

In [110]:
print(train_labels.dtype)
epochs = 50
logistic_model = nn.Sequential(
                    nn.Linear(hidden_size, 20),
                    nn.Sigmoid(),
                    )
logistic_model.cuda()
dtype = torch.cuda.FloatTensor
logistic_model.type = torch.cuda.FloatTensor
train_features, test_features = torch.tensor(train_features).to(device), torch.tensor(test_features).to(device)
train_labels, test_labels = torch.tensor(train_labels).to(device), torch.tensor(test_labels).to(device)
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(logistic_model.parameters(), lr=0.0001)

for epoch in range(epochs):
  optimizer.zero_grad()
  output = logistic_model(train_features)
  loss = criterion(output, train_labels)
  print("Epoch {0} with training loss: {1}".format(epoch, loss))
  loss.backward()
  optimizer.step()    

torch.float32
Epoch 0 with training loss: 0.8349576592445374
Epoch 1 with training loss: 0.8344990015029907
Epoch 2 with training loss: 0.834040641784668
Epoch 3 with training loss: 0.8335826396942139
Epoch 4 with training loss: 0.8331249356269836
Epoch 5 with training loss: 0.8326676487922668
Epoch 6 with training loss: 0.8322106599807739
Epoch 7 with training loss: 0.8317540287971497
Epoch 8 with training loss: 0.8312978744506836
Epoch 9 with training loss: 0.8308420181274414
Epoch 10 with training loss: 0.8303865790367126
Epoch 11 with training loss: 0.8299315571784973
Epoch 12 with training loss: 0.8294769525527954
Epoch 13 with training loss: 0.8290228247642517
Epoch 14 with training loss: 0.8285690546035767
Epoch 15 with training loss: 0.8281158804893494
Epoch 16 with training loss: 0.8276630640029907
Epoch 17 with training loss: 0.8272107243537903
Epoch 18 with training loss: 0.826758861541748
Epoch 19 with training loss: 0.8263075351715088
Epoch 20 with training loss: 0.8258567

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


In [127]:
def convert_to_thresholds(data):
  return data>0.5

def test_model():
  preds = logistic_model(test_features).cpu().data.numpy()
  correct = test_labels.cpu().numpy()
  preds = convert_to_thresholds(preds)
  print("F1 Score: ", f1_score(preds, correct, average="micro"))
  print("ROC-AUC: ", roc_auc_score(correct, preds))

test_model()

F1 Score:  0.5273833671399594
ROC-AUC:  0.4965403661137956
